In [ ]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Add the path to access crud_operations.py
import sys
sys.path.append('/home/aerieldenmark_snhu/Desktop/6-1_Milestone_Submission')

# Import AnimalShelter from the CRUD module
from crud_operations import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# Update with your username and password
username = "aacuser"
password = "CS340module3"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# Retrieve data from MongoDB
try:
    df = pd.DataFrame.from_records(db.read({}))
    df.drop(columns=['_id'], inplace=True, errors='ignore')  # Drop '_id' column if it exists
    print("DataFrame loaded successfully with columns:", df.columns)
except Exception as e:
    print(f"Error loading DataFrame: {e}")
    df = pd.DataFrame()  # Load an empty DataFrame if errors occur

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Encode Grazioso Salvare’s logo
image_filename = '/home/aerieldenmark_snhu/Desktop/Grazioso Salvare Logo.png'
encoded_image = None
try:
    with open(image_filename, 'rb') as image_file:
        encoded_image = base64.b64encode(image_file.read()).decode()
except FileNotFoundError:
    print("Logo file not found. Ensure the correct file path.")

# App layout
app.layout = html.Div([
    # Logo and title
    html.Div([
        html.Img(src=f'data:image/png;base64,{encoded_image}', style={'height': '100px'}) if encoded_image else html.H3("Logo Missing"),
        html.Center(html.B(html.H1('CS-340 Dashboard'))),
        html.Center(html.P("Developed by Aeriel Denmark", style={'fontSize': '18px', 'fontStyle': 'italic'})),  # Unique identifier
    ]),
    html.Hr(),

    # Interactive filter options
    html.Div([
        html.Label('Filter Rescue Type:'),
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'All', 'value': 'ALL'},
                {'label': 'Water Rescue', 'value': 'WATER'},
                {'label': 'Mountain/Wilderness Rescue', 'value': 'MOUNTAIN'},
                {'label': 'Disaster/Individual Tracking', 'value': 'DISASTER'}
            ],
            value='ALL',  # Default selection
            inline=True
        ),
    ]),
    html.Hr(),

    # Data Table
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,  # Display 10 rows per page
        style_table={'overflowX': 'auto'},  # Ensure the table fits the screen width
        sort_action="native",  # Enable sorting
        filter_action="native",  # Enable filtering
        row_selectable="single",  # Allow single row selection
        selected_rows=[],  # Default: No row selected
    ),
    html.Br(),
    html.Hr(),

    # Charts and Map
    html.Div(className='row',
             style={'display': 'flex'},
             children=[
                 html.Div(
                     id='graph-id',
                     className='col s12 m6',
                 ),
                 html.Div(
                     id='map-id',
                     className='col s12 m6',
                 )
             ])
])

#############################################
# Interaction Between Components / Controller
#############################################

# Callback to update the data table based on the selected filter
@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    print(f"Filter selected: {filter_type}")  # Debugging
    try:
        if filter_type == 'WATER':
            filtered_data = db.get_water_rescue_dogs()
        elif filter_type == 'MOUNTAIN':
            filtered_data = db.get_mountain_rescue_dogs()
        elif filter_type == 'DISASTER':
            filtered_data = db.get_disaster_rescue_dogs()
        else:
            filtered_data = db.read({})
        
        print(f"Filtered data length: {len(filtered_data)}")  # Debugging
        return pd.DataFrame.from_records(filtered_data).to_dict('records')
    except Exception as e:
        print(f"Error in update_dashboard: {e}")
        return []

# Callback to update the geolocation map
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    print(f"ViewData received: {viewData[:5] if viewData else 'None'}")  # Debugging
    print(f"Selected index: {index}")  # Debugging

    if not viewData or len(viewData) == 0:
        return [html.P("No data available for map")]

    dff = pd.DataFrame.from_dict(viewData)

    # Handle selected row index
    if index is None or len(index) == 0:
        row = 0  # Default to the first row
    else:
        row = index[0]

    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[dff.iloc[row]['location_lat'], dff.iloc[row]['location_long']],
                      children=[
                          dl.Tooltip(dff.iloc[row]['breed']),
                          dl.Popup([
                              html.H1("Animal Details"),
                              html.P(f"Name: {dff.iloc[row]['name']}"),
                              html.P(f"Breed: {dff.iloc[row]['breed']}"),
                              html.P(f"Age: {dff.iloc[row]['age_upon_outcome']}")
                          ])
                      ])
        ])
    ]

# Callback to update the pie chart based on the selected filter
@app.callback(
    Output('graph-id', "children"),
    [Input('filter-type', 'value')]
)
def update_pie_chart(filter_type):
    print(f"Filter selected for pie chart: {filter_type}")  # Debugging
    try:
        # Fetch filtered data
        if filter_type == 'WATER':
            filtered_data = db.get_water_rescue_dogs()
        elif filter_type == 'MOUNTAIN':
            filtered_data = db.get_mountain_rescue_dogs()
        elif filter_type == 'DISASTER':
            filtered_data = db.get_disaster_rescue_dogs()
        else:
            filtered_data = db.read({})
        
        # Convert to DataFrame
        filtered_df = pd.DataFrame.from_records(filtered_data)
        print(f"Filtered data length for pie chart: {len(filtered_df)}")  # Debugging

        # Create pie chart for breed distribution
        if not filtered_df.empty and 'breed' in filtered_df.columns:
            fig = px.pie(filtered_df, names='breed', title='Breed Distribution')
            return [dcc.Graph(figure=fig)]
        else:
            return [html.P("No data available for graph")]
    except Exception as e:
        print(f"Error in update_pie_chart: {e}")
        return [html.P("Error updating pie chart")]

# Run the Dash app
app.run_server(debug=True)